In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import os

In [4]:
dataDir = '../data'

### topic word embedding

In [5]:
topic_embeds = pd.read_csv(os.path.join(dataDir, 'topic_vectors_64d.txt'),
                        names = ['topic_id', 'vector'],
                        usecols = ['topic_id', ],
                        index_col= 'topic_id',
                        sep = '\t',
                        squeeze= True)

In [6]:
num_topics = len(topic_embeds)

In [7]:
topic_vec = np.load(os.path.join(dataDir, 'topic_array.npy'))

In [8]:
topic_vec.dtype

dtype('float64')

In [9]:
topic_idmap = pd.Series(np.int64(np.arange(len(topic_embeds))), index = topic_embeds.index)

In [10]:
topic_idmap['-1'] = 10000

## 处理用户属性

In [19]:
user_df = pd.read_csv(os.path.join(dataDir, 'member_info_1020.csv'),
                              #names= ['user_id', 'gender', 'visit_freq', 'binary_A',
                               #      'binary_B', 'binary_C', 'binary_D', 'binary_E',
                                #     'category_A' 'category_B', 'category_C',
                                 #    'category_D', 'category_E', 'salt', 'follow_topics', 'interest_topics'],
                              #sep= ',',
                      index_col= 'user_id',
                      # nrows= 5000
                     )

In [20]:
user_df.head()

,gender,visit_freq,binary_A,binary_B,binary_C,binary_D,binary_E,category_A,category_B,category_C,category_D,category_E,salt_value,follow_topics,interest_topics
user_id,,,,,,,,,,,,,,,
M1934753188,male,monthly,0,1,0,1,0,MD470265,BR470265,PV929066,CT929066,PF470265,764,T540,"T21107:1.7915097,T405:1.6123838,T4436:1.518003..."
M595924114,male,daily,0,0,0,1,1,Others,BR596936,PV002320,CT840234,PF470265,671,"T44126,T15940,T839,T8978,T2934,T1113,T3914,T12...","T18016:2.0650618,T2384:1.2503042,T1142:1.13569..."
M1473482940,female,weekly,0,1,0,1,0,MD116493,BR641329,PV170953,CT470265,PF470265,454,"T30874,T2113,T8656,T21,T523,T8,T116,T5727,T68,...","T46:1.330939,T2159:1.1296458,T379:1.1241927,T1..."
M578477092,male,daily,1,1,0,1,0,MD889589,BR803759,PV545833,CT545833,PF470265,588,"T946,T7323,T297,T2660,T36067,T53107,T2654,T507...","T15918:1.9479566,T8106:1.8578106,T4787:1.58486..."
M1088794709,male,weekly,0,1,0,0,0,MD825760,BR641329,PV071037,CT470265,PF470265,361,"T582,T558,T28776,T5186,T9081,T2419,T2693,T2299...","T65:1.5992582,T867:1.3179373,T93:1.2095009,T31..."


In [21]:
user_df.describe()

,binary_A,binary_B,binary_C,binary_D,binary_E,salt_value
count,1.931654e+06,1.931654e+06,1.931654e+06,1.931654e+06,1.931654e+06,1.931654e+06
mean,5.973637e-01,3.117147e-01,2.953117e-02,1.475352e-01,5.420381e-02,3.466341e+02
std,4.904288e-01,4.631941e-01,1.692900e-01,3.546388e-01,2.264195e-01,8.260136e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.500000e+01
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.950000e+02
50%,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.200000e+02
75%,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.730000e+02
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,8.900000e+02


#### -1 对应的　topic 以加权均值填充

In [23]:
tp_count = np.zeros((num_topics, 1))
for tp_s in user_df['follow_topics']:
    if tp_s != '-1':
        id_list = tp_s.split(',')
        ind_list = topic_idmap[id_list].tolist()
        for ind in ind_list:
            tp_count[ind] += 1

In [24]:
tp_weighted_mean = np.max(np.multiply(topic_vec, tp_count), axis= 0) / np.sum(tp_count)

In [25]:
tp_weighted_mean.shape

(64,)

In [26]:
def maxpool_topics(s):
    try:
        id_list = s.split(',')
        ind_list = topic_idmap[id_list].tolist()
        topic_array = topic_vec[ind_list]
        topic_mp = np.max(topic_array, axis= 0)
    except:
        topic_mp = tp_weighted_mean
    return topic_mp

In [28]:
def meanpool_topics(s):
    try:
        id_list = s.split(',')
        ind_list = topic_idmap[id_list].tolist()
        topic_array = topic_vec[ind_list]
        topic_mp = np.mean(topic_array, axis= 0)
    except:
        topic_mp = tp_weighted_mean
    return topic_mp

In [29]:
def minpool_topics(s):
    try:
        id_list = s.split(',')
        ind_list = topic_idmap[id_list].tolist()
        topic_array = topic_vec[ind_list]
        topic_mp = np.min(topic_array, axis= 0)
    except:
        topic_mp = tp_weighted_mean
    return topic_mp

#### follow topics 最大池化

In [30]:
user_df['follow_topics_maxpool_vec'] = user_df['follow_topics'].apply(maxpool_topics)

In [32]:
user_df['follow_topics_maxpool_vec'].head()

user_id
M1934753188    [6.125305, -1.4180568, -1.3143845, -2.9268239,...
M595924114     [3.8494484, 3.977306, 5.7516003, 4.9546337, 4....
M1473482940    [3.4723787, 4.6616564, 3.5893476, 5.427303, 5....
M578477092     [8.753078, 7.646762, 7.8675485, 5.283248, 6.17...
M1088794709    [9.542506, 4.7182026, 4.145554, 2.87201, 7.516...
Name: follow_topics_maxpool_vec, dtype: object

In [33]:
follow_tp_maxpool_array = np.array(user_df['follow_topics_maxpool_vec'].tolist())

In [34]:
follow_tp_maxpool_array.shape

(1931654, 64)

In [35]:
np.save(os.path.join(dataDir, 'user_follow_topics_maxpool.npy'), follow_tp_maxpool_array)

#### follow topic 均值池化

In [36]:
user_df['follow_topics_meanpool_vec'] = user_df['follow_topics'].apply(meanpool_topics)

In [37]:
user_df['follow_topics_meanpool_vec'].head()

user_id
M1934753188    [6.125305, -1.4180568, -1.3143845, -2.9268239,...
M595924114     [0.30575216599999994, -0.9980522930666669, 0.0...
M1473482940    [0.1508028030833333, 0.7434654864583335, -0.25...
M578477092     [1.4605925141999998, -0.33966116570100013, 0.3...
M1088794709    [0.9643650885294115, 0.31516375361764704, 0.82...
Name: follow_topics_meanpool_vec, dtype: object

In [38]:
follow_tp_meanpool_array = np.array(user_df['follow_topics_meanpool_vec'].tolist())

In [39]:
follow_tp_meanpool_array.shape

(1931654, 64)

In [40]:
np.save(os.path.join(dataDir, 'user_follow_topics_meanpool.npy'), follow_tp_meanpool_array)

#### follow topic 最小池化

In [41]:
user_df['follow_topics_minpool_vec'] = user_df['follow_topics'].apply(minpool_topics)

In [42]:
user_df['follow_topics_minpool_vec'].head()

user_id
M1934753188    [6.125305, -1.4180568, -1.3143845, -2.9268239,...
M595924114     [-4.714479, -5.0564847, -6.1172757, -3.2384923...
M1473482940    [-5.314164, -4.025875, -3.3724556, -3.7077088,...
M578477092     [-4.448856, -7.467456, -7.378821, -8.584922, -...
M1088794709    [-5.225668, -5.5874524, -4.6888633, -8.055199,...
Name: follow_topics_minpool_vec, dtype: object

In [43]:
follow_tp_minpool_array = np.array(user_df['follow_topics_minpool_vec'].tolist())

In [44]:
follow_tp_minpool_array.shape

(1931654, 64)

In [45]:
np.save(os.path.join(dataDir, 'user_follow_topics_minpool.npy'), follow_tp_minpool_array)

#### 拼接，保存

In [46]:
user_follow_topics_mp = np.concatenate([follow_tp_maxpool_array, follow_tp_meanpool_array, follow_tp_minpool_array], axis= -1)

In [47]:
np.save(os.path.join(dataDir, 'user_follow_topics_mp.npy'), user_follow_topics_mp)

### 添加topic 对应索引列

### 盐值 salt value归一化

In [75]:
from sklearn.preprocessing import MinMaxScaler

In [76]:
saltValueScaler = MinMaxScaler((0, 1))

In [77]:
user_df['salt_value'] = saltValueScaler.fit_transform(user_df[['salt_value', ]])

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


### 编码类别属性

In [78]:
from sklearn.preprocessing import LabelEncoder

In [79]:
gender_encoder = LabelEncoder()

In [80]:
user_df['gender'] = gender_encoder.fit_transform(user_df['gender'])

In [81]:
visitFreq_encoder = LabelEncoder()

In [82]:
user_df['visit_freq'] = visitFreq_encoder.fit_transform(user_df['visit_freq'])

In [83]:
category_A_encoder = LabelEncoder()

user_df['category_A'] = category_A_encoder.fit_transform(user_df['category_A'])

In [84]:
category_B_encoder = LabelEncoder()

user_df['category_B'] = category_B_encoder.fit_transform(user_df['category_B'])

In [85]:
category_C_encoder = LabelEncoder()

user_df['category_C'] = category_C_encoder.fit_transform(user_df['category_C'])

In [86]:
category_D_encoder = LabelEncoder()

user_df['category_D'] = category_D_encoder.fit_transform(user_df['category_D'])

In [87]:
category_E_encoder = LabelEncoder()

user_df['category_E'] = category_E_encoder.fit_transform(user_df['category_E'])

#### 用户id编码

In [89]:
user_df['user_index'] = np.arange(len(user_df))

In [90]:
len(user_df)

1931654

共 1931654个 用户

#### 统计topic所占权重和

In [131]:
tp_weight = np.zeros((num_topics, 1))
for tp_s in user_df['interest_topics']:
    if tp_s != '-1':
        id_str_list = tp_s.split(',')
        for id_str in id_str_list:
            tp_id, weight_str = id_str.split(':')
            weight = float(weight_str)
            # print(tp_id, weight)
            
            tp_weight[topic_idmap[tp_id]] += weight

In [134]:
tp_weight[:100,]

array([[    0.        ],
       [51948.39888056],
       [33987.26315429],
       [17684.07810108],
       [40966.03172979],
       [22649.40822608],
       [15049.13911903],
       [40835.19418761],
       [48815.12520887],
       [20187.81928158],
       [23216.35223568],
       [13801.42462087],
       [17182.09323776],
       [ 5656.35933362],
       [15851.46695401],
       [27580.26397775],
       [29635.54168217],
       [41146.43601064],
       [34605.12741778],
       [14411.78559699],
       [10410.12467941],
       [29863.90690965],
       [20313.63107364],
       [20513.03201613],
       [20674.17432297],
       [34228.18583081],
       [10657.10852358],
       [32626.13109777],
       [30295.55705368],
       [ 4933.24136651],
       [67045.66956224],
       [17145.51438338],
       [ 8797.63776966],
       [ 8251.64858598],
       [12180.62890914],
       [23916.00224894],
       [21275.02661513],
       [24758.36588506],
       [19960.98524567],
       [16299.62462713],


In [142]:
topic_vec.shape

(100000, 64)

In [143]:
tp_weight.shape

(100000, 1)

In [145]:
tp_weight.dtype

dtype('float64')

#### interest topic 加权和

In [148]:
def map_weighted_topics(tp_s):
    id_str_list = tp_s.split(',')
    tp_list = []
    weight_list = []
    for id_str in id_str_list:
        tp_id, weight_str = id_str.split(':')
        weight = float(weight_str)
        if weight < 100:
            tp_list.append(tp_id)
            weight_list.append(weight)
    tp_inds = topic_idmap[tp_list]
    tp_array = topic_vec[tp_inds]
    weight_array = np.array(weight_list).reshape(1, -1)
    weighted_tp = np.dot(weight_array, tp_array) 
    return weighted_tp

In [149]:
interest_topic_wp = user_df['interest_topics'].loc[user_df['interest_topics'] != '-1'].map(map_weighted_topics)

In [165]:
interest_topic_wp_array = np.array(interest_topic_wp.tolist())

In [166]:
interest_topic_wp_array.shape

(1410201, 1, 64)

In [167]:
interest_topic_mean_vec = np.mean(interest_topic_wp_array, axis= 0)

In [168]:
interest_topic_mean_vec.shape

(1, 64)

In [183]:
interest_topic_wp_full = interest_topic_wp[user_df.index]

In [184]:
interest_topic_wp_full.head(100)

user_id
M1934753188    [[5.255145018431923, 28.50721119338999, 5.7193...
M595924114     [[-19.947685124736022, 13.387408371544119, -3....
M1473482940    [[-2.582439761748401, 6.167275103940158, -0.75...
M578477092     [[14.90885522421449, 14.438666038597825, 11.79...
M1088794709    [[15.879302100003166, -11.453530874701517, -7....
M791147511     [[-13.528778701131753, 0.9688039794237486, 1.5...
M391878471                                                   NaN
M1642994942                                                  NaN
M3475398927    [[9.237489395788591, -5.474324058663583, -12.0...
M3321320610    [[10.877166869215074, -18.71358118595563, -12....
M667025082                                                   NaN
M1702145609                                                  NaN
M48697732                                                    NaN
M1147507896    [[17.19764123280288, 31.86464801467039, -23.38...
M2359456368    [[21.47295947372292, -8.348840217343001, 21.05...
M3359293730    [[

In [185]:
len(interest_topic_wp_full), len(interest_topic_wp)

(1931654, 1410201)

In [188]:
interest_topic_wp_full = interest_topic_wp_full.map(lambda x: interest_topic_mean_vec if np.any(np.isnan(x)) else x)

In [189]:
interest_topic_wp_full.head(100)

user_id
M1934753188    [[5.255145018431923, 28.50721119338999, 5.7193...
M595924114     [[-19.947685124736022, 13.387408371544119, -3....
M1473482940    [[-2.582439761748401, 6.167275103940158, -0.75...
M578477092     [[14.90885522421449, 14.438666038597825, 11.79...
M1088794709    [[15.879302100003166, -11.453530874701517, -7....
M791147511     [[-13.528778701131753, 0.9688039794237486, 1.5...
M391878471     [[2.0622380868806522, -1.0639604998612744, 1.6...
M1642994942    [[2.0622380868806522, -1.0639604998612744, 1.6...
M3475398927    [[9.237489395788591, -5.474324058663583, -12.0...
M3321320610    [[10.877166869215074, -18.71358118595563, -12....
M667025082     [[2.0622380868806522, -1.0639604998612744, 1.6...
M1702145609    [[2.0622380868806522, -1.0639604998612744, 1.6...
M48697732      [[2.0622380868806522, -1.0639604998612744, 1.6...
M1147507896    [[17.19764123280288, 31.86464801467039, -23.38...
M2359456368    [[21.47295947372292, -8.348840217343001, 21.05...
M3359293730    [[

In [192]:
interest_topic_wp_full_array = np.squeeze(np.array(interest_topic_wp_full.tolist()))

In [193]:
interest_topic_wp_full_array.shape

(1931654, 64)

In [196]:
assert interest_topic_wp_full_array.shape[0] == len(user_df)

In [194]:
np.save(os.path.join(dataDir, 'interest_topic_wp.npy'), interest_topic_wp_full_array)

#### 添加 follow_topic_wp 索引列

In [91]:
user_df['interest_topics_wp'] = np.arange(len(user_df))

#### user answer count

In [56]:
user_df['answer_count'] = user_ans_count[user_df.index]

In [57]:
user_df['answer_count'].head(10)

user_id
M1934753188    2.0
M595924114     1.0
M1473482940    1.0
M578477092     2.0
M1088794709    1.0
M791147511     1.0
M391878471     1.0
M1642994942    NaN
M3475398927    4.0
M3321320610    NaN
Name: answer_count, dtype: float64

In [58]:
user_df.loc[user_df['answer_count'].isna(), 'answer_count'] = 0

In [59]:
user_df['answer_count'].head(10)

user_id
M1934753188    2.0
M595924114     1.0
M1473482940    1.0
M578477092     2.0
M1088794709    1.0
M791147511     1.0
M391878471     1.0
M1642994942    0.0
M3475398927    4.0
M3321320610    0.0
Name: answer_count, dtype: float64

In [64]:
user_df['invite_count'] = user_invite_count[user_df.index]

In [65]:
user_df['invite_count'].head(10)

user_id
M1934753188    14.0
M595924114     13.0
M1473482940    20.0
M578477092     21.0
M1088794709     8.0
M791147511      8.0
M391878471      3.0
M1642994942     NaN
M3475398927    26.0
M3321320610     7.0
Name: invite_count, dtype: float64

In [67]:
user_df.loc[user_df['invite_count'].isna(), 'invite_count'] = 1E-10

In [71]:
user_df['accept_ratio'].head(10)

user_id
M1934753188    0.142857
M595924114     0.076923
M1473482940    0.050000
M578477092     0.095238
M1088794709    0.125000
M791147511     0.125000
M391878471     0.333333
M1642994942    0.000000
M3475398927    0.153846
M3321320610    0.000000
Name: accept_ratio, dtype: float64

#### 检查user_df

In [92]:
user_df.head()

,gender,visit_freq,binary_A,binary_B,binary_C,binary_D,binary_E,category_A,category_B,category_C,...,category_E,salt_value,follow_topics,interest_topics,answer_count,invite_count,accept_ratio,follow_topics_mp,user_index,interest_topics_wp
user_id,,,,,,,,,,,,,,,,,,,,,
M1934753188,1,1,0,1,0,1,0,82,69,142,...,1,0.841509,T540,"T21107:1.7915097,T405:1.6123838,T4436:1.518003...",2.0,14.0,0.142857,0,0,0
M595924114,1,0,0,0,0,1,1,149,89,1,...,1,0.724528,"T44126,T15940,T839,T8978,T2934,T1113,T3914,T12...","T18016:2.0650618,T2384:1.2503042,T1142:1.13569...",1.0,13.0,0.076923,1,1,1
M1473482940,0,4,0,1,0,1,0,19,99,31,...,1,0.451572,"T30874,T2113,T8656,T21,T523,T8,T116,T5727,T68,...","T46:1.330939,T2159:1.1296458,T379:1.1241927,T1...",1.0,20.0,0.050000,2,2,2
M578477092,1,0,1,1,0,1,0,134,120,88,...,1,0.620126,"T946,T7323,T297,T2660,T36067,T53107,T2654,T507...","T15918:1.9479566,T8106:1.8578106,T4787:1.58486...",2.0,21.0,0.095238,3,3,3
M1088794709,1,4,0,1,0,0,0,123,99,15,...,1,0.334591,"T582,T558,T28776,T5186,T9081,T2419,T2693,T2299...","T65:1.5992582,T867:1.3179373,T93:1.2095009,T31...",1.0,8.0,0.125000,4,4,4


#### 输出user_df

In [106]:
user_df.to_csv(os.path.join(dataDir, 'member_info_1106.csv'),
#                               columns= ['gender', 'visit_freq', 'binary_A',
#                                     'binary_B', 'binary_C', 'binary_D', 'binary_E',
#                                     'category_A', 'category_B', 'category_C',
#                                     'category_D', 'category_E', 'salt_value', 'follow_topics_mp', 'interest_topics_wp', 'user_index',
#                                        'invite_count', 'answer_count', 'accept_ratio'],
                              sep= '\t',)

### 处理新的用户属性

In [26]:
user_df = pd.read_csv(os.path.join(dataDir, 'member_info_1106.csv'),
#                               columns= ['gender', 'visit_freq', 'binary_A',
#                                     'binary_B', 'binary_C', 'binary_D', 'binary_E',
#                                     'category_A', 'category_B', 'category_C',
#                                     'category_D', 'category_E', 'salt_value', 'follow_topics_mp', 'interest_topics_wp', 'user_index',
#                                        'invite_count', 'answer_count', 'accept_ratio'],
                      index_col = 0,
                              sep= '\t',)

In [17]:
user_df.head()

,gender,visit_freq,binary_A,binary_B,binary_C,binary_D,binary_E,category_A,category_B,category_C,category_D,category_E,salt_value,follow_topics_mp,interest_topics_wp,user_index,invite_count,answer_count,accept_ratio
user_id,,,,,,,,,,,,,,,,,,,
M1934753188,1,1,0,1,0,1,0,82,69,142,135,1,0.841509,0,0,0,14.0,2.0,0.142857
M595924114,1,0,0,0,0,1,1,149,89,1,122,1,0.724528,1,1,1,13.0,1.0,0.076923
M1473482940,0,4,0,1,0,1,0,19,99,31,75,1,0.451572,2,2,2,20.0,1.0,0.050000
M578477092,1,0,1,1,0,1,0,134,120,88,86,1,0.620126,3,3,3,21.0,2.0,0.095238
M1088794709,1,4,0,1,0,0,0,123,99,15,75,1,0.334591,4,4,4,8.0,1.0,0.125000


In [28]:
for feat in ['gender', 'visit_freq','category_A','category_B','category_C','category_D','category_E']:
    col_name = '{}_count'.format(feat)
    user_df[col_name] = user_df[feat].map(user_df.groupby(feat)['invite_count'].sum().astype(int))
#     user_df.loc[user_df[col_name] < 2, feat] = -1
#     user_df[feat] += 1
    user_df[col_name] = user_df[feat].map(user_df[feat].value_counts().astype(int))
    user_df[col_name] = (user_df[col_name] - user_df[col_name].min()) / (user_df[col_name].max() - user_df[col_name].min())



In [30]:
user_df.columns

Index(['gender', 'visit_freq', 'binary_A', 'binary_B', 'binary_C', 'binary_D',
       'binary_E', 'category_A', 'category_B', 'category_C', 'category_D',
       'category_E', 'salt_value', 'follow_topics_mp', 'interest_topics_wp',
       'user_index', 'invite_count', 'answer_count', 'accept_ratio',
       'gender_count', 'visit_freq_count', 'category_A_count',
       'category_B_count', 'category_C_count', 'category_D_count',
       'category_E_count'],
      dtype='object')

In [29]:
user_df.head(20)

,gender,visit_freq,binary_A,binary_B,binary_C,binary_D,binary_E,category_A,category_B,category_C,...,invite_count,answer_count,accept_ratio,gender_count,visit_freq_count,category_A_count,category_B_count,category_C_count,category_D_count,category_E_count
user_id,,,,,,,,,,,,,,,,,,,,,
M1934753188,1,1,0,1,0,1,0,82,69,142,...,1.400000e+01,2.0,0.142857,0.000000,0.542701,1.000000,1.000000,0.413872,0.115038,1.0
M595924114,1,0,0,0,0,1,1,149,89,1,...,1.300000e+01,1.0,0.076923,0.000000,0.451365,0.104223,0.000032,0.437643,0.001159,1.0
M1473482940,0,4,0,1,0,1,0,19,99,31,...,2.000000e+01,1.0,0.050000,0.130213,1.000000,0.062218,0.415082,0.024786,1.000000,1.0
M578477092,1,0,1,1,0,1,0,134,120,88,...,2.100000e+01,2.0,0.095238,0.000000,0.451365,0.005084,0.124367,0.521151,0.145250,1.0
M1088794709,1,4,0,1,0,0,0,123,99,15,...,8.000000e+00,1.0,0.125000,0.000000,1.000000,0.023493,0.415082,0.291936,1.000000,1.0
M791147511,1,0,1,0,1,1,1,132,118,142,...,8.000000e+00,1.0,0.125000,0.000000,0.451365,0.007676,0.092387,0.413872,0.115038,1.0
M391878471,0,3,0,0,0,0,0,82,69,79,...,3.000000e+00,1.0,0.333333,0.130213,0.049347,1.000000,1.000000,0.077418,1.000000,1.0
M1642994942,2,1,1,0,0,0,0,82,69,15,...,1.000000e-10,0.0,0.000000,1.000000,0.542701,1.000000,1.000000,0.291936,1.000000,1.0
M3475398927,0,0,1,0,0,0,0,117,27,95,...,2.600000e+01,4.0,0.153846,0.130213,0.451365,0.000216,0.072487,0.521275,0.041416,1.0


In [31]:
user_df.to_csv(os.path.join(dataDir, 'member_info_1109.csv'),
                              sep= '\t',)

### 1112

In [49]:
user_df = pd.read_csv(os.path.join(dataDir, 'member_info_1109.csv'),
                              #names= ['user_id', 'gender', 'visit_freq', 'binary_A',
                               #      'binary_B', 'binary_C', 'binary_D', 'binary_E',
                                #     'category_A' 'category_B', 'category_C',
                                 #    'category_D', 'category_E', 'salt', 'follow_topics', 'interest_topics'],
                              #sep= ',',
                      index_col= 'user_id',
#                       nrows= 5000
                      sep= '\t'
                     )

In [50]:
user_df.head()

,gender,visit_freq,binary_A,binary_B,binary_C,binary_D,binary_E,category_A,category_B,category_C,...,invite_count,answer_count,accept_ratio,gender_count,visit_freq_count,category_A_count,category_B_count,category_C_count,category_D_count,category_E_count
user_id,,,,,,,,,,,,,,,,,,,,,
M1934753188,1,1,0,1,0,1,0,82,69,142,...,14.0,2.0,0.142857,0.000000,0.542701,1.000000,1.000000,0.413872,0.115038,1.0
M595924114,1,0,0,0,0,1,1,149,89,1,...,13.0,1.0,0.076923,0.000000,0.451365,0.104223,0.000032,0.437643,0.001159,1.0
M1473482940,0,4,0,1,0,1,0,19,99,31,...,20.0,1.0,0.050000,0.130213,1.000000,0.062218,0.415082,0.024786,1.000000,1.0
M578477092,1,0,1,1,0,1,0,134,120,88,...,21.0,2.0,0.095238,0.000000,0.451365,0.005084,0.124367,0.521151,0.145250,1.0
M1088794709,1,4,0,1,0,0,0,123,99,15,...,8.0,1.0,0.125000,0.000000,1.000000,0.023493,0.415082,0.291936,1.000000,1.0


In [51]:
user_df['invite_count_scaled'] = MinMaxScaler().fit_transform(user_df['invite_count'])

NameError: name 'MinMaxScaler' is not defined

In [48]:
user_df.head()

,gender,visit_freq,binary_A,binary_B,binary_C,binary_D,binary_E,category_A,category_B,category_C,category_D,category_E,salt_value,follow_topics,interest_topics,follow_topics_maxpool_vec,follow_topics_meanpool_vec,follow_topics_minpool_vec
user_id,,,,,,,,,,,,,,,,,,
M1934753188,male,monthly,0,1,0,1,0,MD470265,BR470265,PV929066,CT929066,PF470265,764,T540,"T21107:1.7915097,T405:1.6123838,T4436:1.518003...","[6.125305, -1.4180568, -1.3143845, -2.9268239,...","[6.125305, -1.4180568, -1.3143845, -2.9268239,...","[6.125305, -1.4180568, -1.3143845, -2.9268239,..."
M595924114,male,daily,0,0,0,1,1,Others,BR596936,PV002320,CT840234,PF470265,671,"T44126,T15940,T839,T8978,T2934,T1113,T3914,T12...","T18016:2.0650618,T2384:1.2503042,T1142:1.13569...","[3.8494484, 3.977306, 5.7516003, 4.9546337, 4....","[0.30575216599999994, -0.9980522930666669, 0.0...","[-4.714479, -5.0564847, -6.1172757, -3.2384923..."
M1473482940,female,weekly,0,1,0,1,0,MD116493,BR641329,PV170953,CT470265,PF470265,454,"T30874,T2113,T8656,T21,T523,T8,T116,T5727,T68,...","T46:1.330939,T2159:1.1296458,T379:1.1241927,T1...","[3.4723787, 4.6616564, 3.5893476, 5.427303, 5....","[0.1508028030833333, 0.7434654864583335, -0.25...","[-5.314164, -4.025875, -3.3724556, -3.7077088,..."
M578477092,male,daily,1,1,0,1,0,MD889589,BR803759,PV545833,CT545833,PF470265,588,"T946,T7323,T297,T2660,T36067,T53107,T2654,T507...","T15918:1.9479566,T8106:1.8578106,T4787:1.58486...","[8.753078, 7.646762, 7.8675485, 5.283248, 6.17...","[1.4605925141999998, -0.33966116570100013, 0.3...","[-4.448856, -7.467456, -7.378821, -8.584922, -..."
M1088794709,male,weekly,0,1,0,0,0,MD825760,BR641329,PV071037,CT470265,PF470265,361,"T582,T558,T28776,T5186,T9081,T2419,T2693,T2299...","T65:1.5992582,T867:1.3179373,T93:1.2095009,T31...","[9.542506, 4.7182026, 4.145554, 2.87201, 7.516...","[0.9643650885294115, 0.31516375361764704, 0.82...","[-5.225668, -5.5874524, -4.6888633, -8.055199,..."
